 <font size="15">
SKT012.05.01.01.05_scATACseqChiouT2D_DeepLearning-TrainingModelStrideSize50Chromosomes_NewLibraryVersion-ExecutableScript
</font> 

# Training the model

## Import Packages

In [1]:
print(shap.__version__)
print(keras.__version__)

NameError: name 'shap' is not defined

In [15]:
import sys
import optparse
from array import *

import tensorflow
import numpy as np
import pandas as pd

import matplotlib
#matplotlib.use('pdf')
import matplotlib.pyplot as plt

import sklearn
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import KFold


from keras.models import Sequential
from keras.models import model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Bidirectional, Concatenate, PReLU 
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers
#from keras.layers.recurrent import LSTM, Bidirectional, TimeDistributed
from keras.layers import LSTM, Bidirectional, TimeDistributed
from keras.layers import Layer, average, Input
from keras.models import Model
from keras.utils import plot_model

## Defining essential functions

In [16]:
def get_output(input_layer, hidden_layers):
    output = input_layer
    for hidden_layer in hidden_layers: 
        output = hidden_layer(output)
    return output

The function get_output(input_layer, hidden_layers) takes in two arguments, input_layer and hidden_layers. The input_layer is the first layer of a neural network, and hidden_layers is a list of other layers in the network (i.e. the layers that come after the input layer and before the output layer). The function iterates through the hidden_layers and applies each layer to the input_layer one by one. The output of each hidden layer is passed as the input to the next hidden layer. The final output of the function is the output after passing the input through all the hidden layers. Essentially, this function is passing the input through a sequence of layers, and returning the final output after passing through all the layers.

In [17]:
# Define your Model for Deep Learning
def build_model():
    forward_input = Input(shape=seq_shape)
    reverse_input = Input(shape=seq_shape)

    hidden_layers = [
        Conv1D(1024, kernel_size=24, padding="valid",
               activation='relu', kernel_initializer='random_uniform'),
        MaxPooling1D(pool_size=10, strides=10, padding='valid'), 
        Dropout(0.2),
        TimeDistributed(Dense(1024, activation='relu')),
        Bidirectional(LSTM(1024, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)),
        Dropout(0.2),
        Flatten(),
        Dense(2048, activation='relu'),
        Dropout(0.4),
        Dense(len(selected_classes), activation='sigmoid')]
    
    forward_output = get_output(forward_input, hidden_layers)     
    reverse_output = get_output(reverse_input, hidden_layers)
    output = average([forward_output, reverse_output])
    model = Model(inputs=[forward_input, reverse_input], outputs=output)

    model.summary()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

This function is defining a neural network model using the Keras library in Python. The model takes two inputs, "forward_input" and "reverse_input", which have the shape specified by the "seq_shape" variable. The model has several layers:

* The first layer is a 1D convolutional layer with 128 filters, a kernel size of 20, and a "valid" padding. It uses the ReLU activation function and a random uniform initialization for the kernel weights.
* The second layer is a max-pooling layer, with a pool size of 10 and stride of 10, using "valid" padding.
* The third layer is a dropout layer with a dropout rate of 0.2, which is used to prevent overfitting.
* The fourth layer is a time-distributed dense layer with 128 units and a ReLU activation function.
* The fifth layer is a bidirectional LSTM layer with 128 units and dropout rate of 0.1 and recurrent dropout rate of 0.1, with return_sequence=True
* The sixth layer is a dropout layer with a dropout rate of 0.2
* The seventh layer is a flatten layer
* The eighth layer is a dense layer with 256 units and a ReLU activation function
* The nineth layer is a dropout layer with a dropout rate of 0.4
* The final layer is a dense layer with the number of units equal to the length of the selected_classes variable, and a sigmoid activation function.

The forward_output and reverse_output are obtained by passing their corresponding input through the hidden layers defined above. The final output of the model is obtained by taking the average of these two outputs. The model is then compiled with the Adam optimizer, binary cross-entropy loss function and accuracy metrics. The summary of the model is also printed out.

In [18]:
def readfile(filename):
    ids = []
    ids_d = {}
    seqs = {}
    classes = {}
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    seq = []
    for line in lines:
        if line[0] == '>':
            ids.append(line[1:].rstrip('\n'))
            if line[1:].rstrip('\n').split('_')[0] not in seqs:
                seqs[line[1:].rstrip('\n').split('_')[0]] = []
            if line[1:].rstrip('\n').split('_')[0] not in ids_d:
                ids_d[line[1:].rstrip('\n').split('_')[0]] = line[1:].rstrip('\n').split('_')[0]
            if line[1:].rstrip('\n').split('_')[0] not in classes:
                classes[line[1:].rstrip('\n').split('_')[0]] = np.zeros(NUM_CLASSES)
            classes[line[1:].rstrip('\n').split('_')[0]][int(line[1:].rstrip('\n').split('_')[1])-1] = 1        
            if seq != []: seqs[ids[-2].split('_')[0]]= ("".join(seq))
            seq = []
        else:
            seq.append(line.rstrip('\n').upper())
    if seq != []:
        seqs[ids[-1].split('_')[0]]=("".join(seq))
    return ids,ids_d,seqs,classes

def one_hot_encode_along_row_axis(sequence):
    to_return = np.zeros((1,len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return[0],
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1

# Create plots for Accuracy or Loss for Training and Validation set for each epoch
def create_plots(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig(foldername + 'accuracy.png')
    plt.clf()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig(foldername + 'loss.png')
    plt.clf()

# Create Excel-file for Accuracy and Loss for Training and Validation set for each epoch
def create_excel(history):
    history_df = pd.DataFrame(history.history)
    history_df.index.name = 'Epoch'
    excel_file = foldername + 'Accuracy_and_Loss.xlsx'
    history_df.to_excel(excel_file)
    
def json_hdf5_to_model(json_filename, hdf5_filename):  
    with open(json_filename, 'r') as f:
        model = model_from_json(f.read())
    model.load_weights(hdf5_filename)
    return model

def loc_to_model_loss(loc):
    return json_hdf5_to_model(loc + 'model.json', loc + 'model_best_loss.hdf5')


# randomly shuffle the order of labels in a dataset for cross-validation, training or testing
def shuffle_label(label):
    for i in range(len(label.T)):
        label.T[i] = shuffle(label.T[i])
    return label

def calculate_roc_pr(score, label):
    output = np.zeros((len(label.T), 2))
    for i in range(len(label.T)):
        roc_ = roc_auc_score(label.T[i], score.T[i])
        pr_ = average_precision_score(label.T[i], score.T[i])
        output[i] = [roc_, pr_]
    return output

## Input data

In [19]:
# Define your input data
NUM_CLASSES = 97
selected_classes = np.arange(NUM_CLASSES)
SEQ_LEN = 500
SEQ_DIM = 4
seq_shape = (SEQ_LEN, SEQ_DIM)

# Define working folders
InputFolder = '/lustre1/project/stg_00041/SanKit/SKT012_Chiou_snATAC_Pancreas/Data/DeepLearning/InputFileHumanPancreas/'
foldername = '/lustre1/project/stg_00041/SanKit/SKT012_Chiou_snATAC_Pancreas/Data/DeepLearning/OutputFileHumanPancreas/TrainedModelStrideSize50Chromosomes/NewVersion/'


# Define your Training and Validation dataset
train_filename = InputFolder + 'MergedFile_WoChr2_WoChr11_extendedStrideSize50.fa'
valid_filename = InputFolder + 'MergedFile_Chr2.fa'


# Define the path for saving models
PATH_TO_SAVE_ARC = foldername + 'model.json'
PATH_TO_SAVE_BEST_LOST_WEIGHTS = foldername + 'model_best_loss.hdf5'
PATH_TO_SAVE_BEST_ACC_WEIGHTS = foldername + 'model_best_acc.hdf5'
PATH_TO_SAVE_END_WEIGHTS = foldername + 'model_end.hdf5'

# Prepare your Training and Validation input
print("Prepare input...")
train_ids, train_ids_d, train_seqs, train_classes = readfile(train_filename)
X_train = np.array([one_hot_encode_along_row_axis(train_seqs[id]) for id in train_ids_d]).squeeze(axis=1)
y_train = np.array([train_classes[id] for id in train_ids_d])
y_train = y_train[:,selected_classes]
X_train = X_train[y_train.sum(axis=1)>0]
y_train = y_train[y_train.sum(axis=1)>0]
X_train_rc = X_train[:,::-1,::-1]
train_data = [X_train, X_train_rc]

valid_ids, valid_ids_d, valid_seqs, valid_classes = readfile(valid_filename)
X_valid = np.array([one_hot_encode_along_row_axis(valid_seqs[id]) for id in valid_ids_d]).squeeze(axis=1)
y_valid = np.array([valid_classes[id] for id in valid_ids_d])
y_valid = y_valid[:,selected_classes]
X_valid = X_valid[y_valid.sum(axis=1)>0]
valid_data = y_valid[y_valid.sum(axis=1)>0]
X_valid_rc = X_valid[:,::-1,::-1]
valid_data = [X_valid, X_valid_rc]

Prepare input...


## Train the model

In [34]:
# Define your model
print('-------------------------------------------')
print("Compile model...")
model = build_model()

model_json = model.to_json()
with open(PATH_TO_SAVE_ARC, "w") as json_file:
    json_file.write(model_json)
    


# Save the weights of the model after each EPOCH 
# if it has the best validation loss or validation accuracy    
print('-------------------------------------------')
print("Model architecture saved to..", PATH_TO_SAVE_ARC)
checkpoint1 = ModelCheckpoint(PATH_TO_SAVE_BEST_LOST_WEIGHTS,
                              monitor='val_loss', verbose=1,
                              save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(PATH_TO_SAVE_BEST_ACC_WEIGHTS,
                              monitor='val_accuracy', verbose=1,
                              save_best_only=True, mode='max')
checkpoint3 = EarlyStopping(monitor='val_loss', patience=6)
callbacks_list = [checkpoint1, checkpoint2, checkpoint3]


# Define parameters
EPOCH = 50
BATCH = 128


# Train Model
print('-------------------------------------------')
print("Train model...")
history = model.fit( train_data, y_train, epochs=EPOCH,
                    batch_size=BATCH, shuffle=True,
                    validation_data=(valid_data, y_valid),
                    verbose=1, callbacks= callbacks_list)


# Plot Accuracy and Loss information
print('-------------------------------------------')
print("Save Plots...")
create_plots(history)
# Save Accuracy and Loss as Excel-file
print('-------------------------------------------')
print("Save Excel...")
create_excel(history)



model.save_weights(PATH_TO_SAVE_END_WEIGHTS)
print("Model weights saved to..", PATH_TO_SAVE_END_WEIGHTS)
plot_model(model, to_file=foldername + 'model.png')

score, acc = model.evaluate(valid_data, y_valid, batch_size=BATCH)
print('Test score:', score)
print('Validation accuracy:', acc)  

-------------------------------------------
Compile model...
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 500, 4)]     0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 500, 4)]     0           []                               
                                                                                                  
 conv1d_3 (Conv1D)              (None, 477, 1024)    99328       ['input_7[0][0]',                
                                                                  'input_8[0][0]']                
                                                                                                  
 max_pooling1d_3 (MaxPooling1D)

KeyboardInterrupt: 